In [2]:
import time
import math
import re
from textblob import TextBlob
import pandas as pd

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim

In [5]:
directory = '~/Datasets/32018/'
#file = 'jeep.txt'
file = 'webhose_cat.pkl'
path = directory + file

In [7]:
df = pd.read_pickle(path)

In [8]:
df.head()

,crawled,language,text,title,url
0,2018-01-30T18:28:45.012+02:00,english,Avery Dennison's (AVY) Q4 results are likely t...,IRobot downgraded to neutral from buy at Sidot...,http://omgili.com/ri/.wHSUbtEfZQRfU.5KUm1RkeXy...
1,2018-01-30T18:29:07.001+02:00,french,"1m95, c’est trop grand. Et sa stature, Bertran...","""Bertrand Zibi Abeghe, encore prisonnier, et t...",http://omgili.com/ri/.wHSUbtEfZTpzFtnXyQJIwJ.j...
2,2018-01-30T18:29:40.000+02:00,english,Tuggers and Topper Industrial Carts Help Trans...,Tuggers and Topper Industrial Carts Help Trans...,http://omgili.com/ri/jHIAmI4hxg.zDiulpymXqU_n4...
3,2018-01-30T18:30:05.007+02:00,english,Currently adding the following games:\n100 (by...,,http://omgili.com/ri/.0rSU5LtMgyggHgoOVy9TMDWT...
4,2018-01-30T18:30:05.013+02:00,english,Quote: : » Currently adding the following game...,,http://omgili.com/ri/.0rSU5LtMgyggHgoOVy9TMDWT...


In [12]:
df = df[df.language=='english']

In [13]:
df['text_clean'] = df['text'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))

In [14]:
df[['text', 'text_clean']].head(5)

,text,text_clean
0,Avery Dennison's (AVY) Q4 results are likely t...,Avery Dennisons AVY Q4 results are likely to g...
2,Tuggers and Topper Industrial Carts Help Trans...,Tuggers and Topper Industrial Carts Help Trans...
3,Currently adding the following games:\n100 (by...,Currently adding the following games:100 by ev...
4,Quote: : » Currently adding the following game...,Quote: : Currently adding the following games...
5,Quote: : » Currently adding the following game...,Quote: : Currently adding the following games...


In [17]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sshepa74/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [20]:
news_clean = [clean(doc).split() for doc in df.text_clean.to_list()]

In [25]:
dictionary = corpora.Dictionary(news_clean)

In [26]:
%time doc_term_matrix = [dictionary.doc2bow(doc) for doc in news_clean]

CPU times: user 34.4 ms, sys: 3.23 ms, total: 37.7 ms
Wall time: 35.8 ms


In [35]:
%time ldamodel3 = LdaMulticore(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
%time ldamodel5 = LdaMulticore(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)
%time ldamodel7 = LdaMulticore(doc_term_matrix, num_topics=7, id2word = dictionary, passes=50)
%time ldamodel9 = LdaMulticore(doc_term_matrix, num_topics=9, id2word = dictionary, passes=50)

CPU times: user 11.4 s, sys: 601 ms, total: 12 s
Wall time: 13.3 s
CPU times: user 15.2 s, sys: 1.15 s, total: 16.4 s
Wall time: 14.6 s
CPU times: user 17 s, sys: 1.47 s, total: 18.5 s
Wall time: 12.1 s
CPU times: user 17.5 s, sys: 1.53 s, total: 19 s
Wall time: 14 s


In [39]:
print(*ldamodel3.print_topics(num_topics=3, num_words=5), sep='\n')
print()
print(*ldamodel5.print_topics(num_topics=5, num_words=5), sep='\n')
print()
print(*ldamodel7.print_topics(num_topics=7, num_words=5), sep='\n')
print()
print(*ldamodel9.print_topics(num_topics=9, num_words=5), sep='\n')

(0, '0.009*"tax" + 0.008*"u" + 0.008*"jan" + 0.007*"company" + 0.006*"share"')
(1, '0.016*"market" + 0.009*"plant" + 0.006*"case" + 0.005*"industry" + 0.005*"product"')
(2, '0.007*"city" + 0.007*"median" + 0.006*"estimate" + 0.006*"university" + 0.006*"2017"')

(0, '0.011*"plant" + 0.007*"case" + 0.007*"city" + 0.006*"median" + 0.006*"estimate"')
(1, '0.016*"caterpillar" + 0.011*"company" + 0.011*"share" + 0.008*"stock" + 0.007*"product"')
(2, '0.014*"inc" + 0.009*"jan" + 0.006*"bucket" + 0.005*"corp" + 0.005*"dec"')
(3, '0.008*"company" + 0.007*"iot" + 0.006*"new" + 0.005*"health" + 0.004*"mnubo"')
(4, '0.015*"market" + 0.011*"tax" + 0.009*"u" + 0.006*"jan" + 0.006*"company"')

(0, '0.007*"new" + 0.006*"company" + 0.006*"house" + 0.006*"market" + 0.005*"system"')
(1, '0.018*"plant" + 0.012*"caterpillar" + 0.011*"case" + 0.009*"share" + 0.008*"company"')
(2, '0.021*"sphere" + 0.020*"amazon" + 0.019*"seattle" + 0.011*"monday" + 0.011*"grand"')
(3, '0.016*"tax" + 0.012*"u" + 0.007*"compa